In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, 3, Finished, Available, Finished)

In [ ]:
import json
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, 4, Finished, Available, Finished)

In [ ]:
#Remove this before running data factory pipeline
#start_date = date.today() -timedelta(7) #7days

StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, 5, Finished, Available, Finished)

In [ ]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline","true").json(f"Files/{start_date}_earthquake_data.json")

StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, 6, Finished, Available, Finished)

NameError: name 'start_date' is not defined

In [ ]:
df

StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, -1, Cancelled, , Cancelled)

In [ ]:
df.head()

StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, -1, Cancelled, , Cancelled)

In [ ]:
#Reshape earhquakedata
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(3).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, -1, Cancelled, , Cancelled)

In [ ]:
#Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, -1, Cancelled, , Cancelled)

In [ ]:
#convert 'time' and 'updated' to timestamp
df = (
    df
    .withColumn('time',(col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, -1, Cancelled, , Cancelled)

In [ ]:
df.head()

StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, -1, Cancelled, , Cancelled)

In [ ]:
#Append to the Silver Table
df.write.mode('append').saveAsTable('earthquake_events_silver')

StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, -1, Cancelled, , Cancelled)

In [ ]:
from notebookutils import mssparkutils

mssparkutils.notebook.exit(json.dumps({
    "start_date": start_date,
    "end_date": end_date
}))

StatementMeta(, 54403366-6f51-4740-8d30-68b47818b2c7, -1, Cancelled, , Cancelled)